In [50]:
# import geojson
import os
import json
import shutil
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, MultiPolygon

In [51]:
# import csv in pandas
df = pd.read_csv('ita_general_2020.csv')

In [52]:
with open('region_shape.geojson') as f:
    data = json.load(f)
    
polygons = gpd.GeoDataFrame.from_features(data['features'])

In [53]:
def get_city_dataset(city_name, region = False, region_name = None):
    
    city = polygons[polygons['prov_name'] == city_name]

    polygon = city['geometry'].iloc[0]

    city_polygon = MultiPolygon(polygon)

    df['in_city'] = df.apply(lambda row: city_polygon.contains(Point(row['longitude'], row['latitude'])), axis=1)

    city_df = df[df['in_city'] == True]

    city_df = city_df.drop(['in_city'], axis=1)
    city_df = city_df.drop(['ita_general_2020'],axis=1)

    city_df['x'] = city_df.apply(lambda row: row['longitude'] - city_df['longitude'].min(), axis=1)
    city_df['y'] = city_df.apply(lambda row: row['latitude'] - city_df['latitude'].min(), axis=1)

    if(region):
        city_df.to_csv(region_name+"_"+city_name+'_xy.csv', index=False)
    else:
        city_df.to_csv(city_name+'_xy.csv', index=False)
    

def get_region_dataset(region_name):
    cities = polygons[polygons['reg_name'] == region_name]['prov_name'].unique()
    print(cities)
    for city in cities:
        get_city_dataset(city, True, region_name)
    path = os.getcwd()
    files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    files = [f for f in files if f.startswith(region_name)]
    print(files)
    df = pd.concat([pd.read_csv(f) for f in files])
    df.to_csv(region_name+'_xy.csv', index=False)
    for f in files:
        os.remove(f)
    path = os.getcwd()
    files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    files = [f for f in files if  f.startswith(region_name) and f.endswith('_xy.csv')]
    print(files)
    for f in files:
        shutil.move(f, './scp_project/src/resources/')

In [ ]:
get_city_dataset('Roma')

In [54]:
get_region_dataset('Lazio')
get_region_dataset('Lombardia')

['Viterbo' 'Rieti' 'Roma' 'Latina' 'Frosinone']
